In [ ]:
import chess
import chess.engine
import chess.pgn
from tqdm import tqdm 

# this was used to have the stockfish model play against itself for data generation

# stockfish engine path
# this is the path on my Home machine it should be chnaged if you are using it 
STOCKFISH_PATH = "/opt/homebrew/bin/stockfish"
stockfish_engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)

# engine stats
stockfish_engine.configure({
    "UCI_LimitStrength": True,
    "UCI_Elo": 2000
})

# Stockfish move
def stockfish_move(board):
    result = stockfish_engine.play(board, chess.engine.Limit(time=0.1, depth= 5))
    return result.move.uci()

# make 20 batches each with a 1000 games
for batch_num in range(20):
    filename = f"simulated_games_batch_{batch_num + 1}.pgn" 
    # open a file
    with open(filename, "w") as pgn_file:
        for game_num in tqdm(range(1000), desc=f"Batch {batch_num + 1}"):
            # set the games and its headers
            board = chess.Board()
            game = chess.pgn.Game()
            game.headers["Event"] = "Simulation"
            game.headers["Site"] = "Local"
            game.headers["Date"] = "2025.08.05"
            game.headers["Round"] = str(game_num + 1)
            game.headers["White"] = "Stockfish_2000"
            game.headers["Black"] = "Stockfish_2000"

            node = game

            #let it play 
            while not board.is_game_over():
                if board.turn == chess.WHITE:
                    move_uci = stockfish_move(board)
                else:
                    move_uci = stockfish_move(board)

                move = chess.Move.from_uci(move_uci)
                board.push(move)
                node = node.add_variation(move)
            # save the outcome and the game 
            outcome = board.outcome()
            if outcome:
                game.headers["Result"] = outcome.result()
            else:
                game.headers["Result"] = "*"

            print(game, file=pgn_file, end="\n\n")

stockfish_engine.quit()